## Load data

In [ ]:
import pandas as pd
loads = pd.read_csv('../data/clean/loads/HighLoads.csv')
loads.head()
loads['total'] = loads.sum(axis=1)
loads.loc[loads['hour'] == 24, 'hour'] = 0
loads.index = pd.to_datetime(loads[['year', 'month', 'day', 'hour']])
last_year = loads.index.year[-1]
loads.loc[loads['hour'] == 0].index += pd.DateOffset(day=1)
#loads.loc[loads['year'] > last_year].index = loads.loc[loads['year'] > last_year].index.year - 1

In [ ]:
import os
import sys

def get_load_data(path='../data/clean/loads', filename='HighLoads.csv', corrections=True, 
                  total=False, *args, **kwargs):
    """ Load consumption data
    
    TODO:
    This could be a csv or it could connect to a DB.
    """
    df = pd.read_csv(os.path.join(path, filename))
    # Calculate the sum of loads
    df['total'] = df.sum(axis=1)
    # Convert to datetime if does not exist
    last_year = df['year'].iloc[-1:].values
    if corrections:
        try:
            df.loc[df['hour'] == 24, 'hour'] = 0
            df.loc[df['hour'] == 0, 'hour'] +=  1
            # Fix below code to represent a year regression
            df.loc[df['year'] > last_year] -= pd.DateOffset(day=365)
        except ValueError as e:
            # TODO Add error if data is wrong
            pass
    df.index = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    
    if total:
        df = df[['total']].sort_index()
    return df.sort_index()

loads = get_load_data()

## Timepoints creation

This is the part where you can change the time resolution of switch. Here you can define the number of timepoints you will use for the analysis. One easy approach, to do it automatically is to groupb the data by maximum per month.

In [ ]:
timepoints = loads.groupby([pd.TimeGrouper('A'), 
                                 pd.TimeGrouper('M')]).idxmax().add_suffix('_peakTime')
timepoints.head()

Using the `timepoints` dataframe we can get the number of timepoints that will be used and the date.

In [ ]:
# Calculate the number of timepoints
ts_num_tps = timepoints.groupby(level=[0]).size().unique()[0]
ts_num_tps

In [ ]:
dates = timepoints['total_peakTime']
dates;

For this example, we will recreate a full day considering the maximum of each month. We will obtain `n`number of points in front and behind the timestamp of the maximum load.

In [ ]:
def get_representative_day(data, dates, number=4):
    """ Construc a representative day based on a single timestamp
    
    Args:
    data
    dates
    number
    Todo: Write readme
    """
    years = []
    if isinstance(dates, pd.Series):
        for day in dates:
            i_date = day - pd.DateOffset(hours=12)
            f_date = day + pd.DateOffset(hours=12)
            mask = (data.index >= i_date) & (data.index <= f_date)
            # reset_index to preserve timepoint reference
            years.append(data.loc[mask].iloc[::number].reset_index())
    else:
        i_date = dates - pd.DateOffset(hours=12)
        f_date = dates + pd.DateOffset(hours=12)
        mask = (data.index >= i_date) & (data.index <= f_date)
        years.append(data.loc[mask].iloc[::number].reset_index())
    output_data = pd.concat(years)
    output_data.rename(columns={'index':'date'}, inplace=True)
    return output_data

output_data = get_representative_day(loads, dates[:6], number=7) 
# Fix the previous line bug (number = tps/day, dates = PeakTimeDay for the first ":#" months)
output_data = output_data.loc[output_data.date.dt.year <= 2025]

## Creating tab file

### Timestamp

The timestamp file needs to include the format:


In [ ]:
identifier = 'P'
output_data['timestamp'] = output_data['date'].dt.strftime('%Y%m%d%H')
output_data['TIMESERIES'] = output_data['date'].dt.strftime('%Y_%m{}'.format(identifier))
output_data['daysinmonth'] = output_data['date'].dt.daysinmonth
output_data;

In [ ]:
timepoints_tab = output_data[['timestamp', 'TIMESERIES', 'daysinmonth']]
timepoints_tab.index.name = 'timepoint_id'
tmp = timepoints_tab.reset_index(drop=True)
tmp = tmp.rename(columns={'TIMESERIES':'timeseries'})
tmp.index += 1  # To start on 1
tmp.index.name = 'timepoint_id'
tmp[['timestamp', 'timeseries']].to_csv('switch-inputs/timepoints.tab', sep='\t')
tmp;

In [ ]:
from collections import OrderedDict
# Todo implement multiple periods based on the data
d = OrderedDict({'INVESTMENT_PERIOD': [2016], 'period_start': [2015], 'period_end':[2025]})
periods_tab = pd.DataFrame(d)
periods_tab= periods_tab.set_index('INVESTMENT_PERIOD')
periods_tab.to_csv('switch-inputs/periods.tab', sep='\t')
periods_tab.head()

In [ ]:
timeseries_tab = timepoints_tab[['TIMESERIES', 'daysinmonth']].drop_duplicates('TIMESERIES').reset_index(drop=True)
ts_duration_of_tp = 6#(24/len(output_data))
timeseries_tab['ts_period'] = 2016 # Fix this to change investment period
timeseries_tab['count'] = timeseries_tab.groupby('ts_period')['TIMESERIES'].transform(len)
timeseries_tab['ts_duration_of_tp'] = ts_duration_of_tp
timeseries_tab['ts_num_tps'] = output_data[['timestamp', 'TIMESERIES']].groupby('TIMESERIES').count().values
timeseries_tab['ts_scale_to_period'] = 10*24*(365/(timeseries_tab['count']))/(timeseries_tab['ts_duration_of_tp']*timeseries_tab['ts_num_tps'])
timeseries_tab.index +=1
timeseries_tab.index.name = 'timepoint_id'
del timeseries_tab['daysinmonth']
del timeseries_tab['count']
timeseries_tab.to_csv('switch-inputs/timeseries.tab', index=False, sep='\t')
timeseries_tab

## Variable capacity factor  data

In [ ]:
import pandas as pd
data_path = '../data/clean/SWITCH/'
ren_cap_data = pd.read_csv(data_path + 'ren-all.csv', index_col=0, parse_dates=True)
ren_cap_data.head()

In [ ]:
renewable_plants = len(ren_cap_data.GENERATION_PROJECT.unique())
renewable_plants

In [ ]:
ren_cap_data_year = ren_cap_data.index.year.unique()
ren_cap_data_year

In [ ]:
periods = set(output_data.date.dt.year)
periods

In [ ]:
# Get the generation using the filter dates
filter_dates = pd.DatetimeIndex(output_data['date'].reset_index(drop=True))
df = pd.DataFrame([])
ren_tmp = ren_cap_data.copy()
ren_tmp.index = ren_tmp.index + pd.DateOffset(years=2)
#df = df.append(ren_tmp)
for year in periods:
    df = df.append(ren_tmp)
    ren_tmp.index = ren_tmp.index + pd.DateOffset(years=1)
grouped = df.loc[filter_dates].dropna().reset_index(drop=True).groupby('GENERATION_PROJECT', as_index=False)
tmp = []
for name, group in grouped:
    tmp.append(group.reset_index(drop=True))
variable_cap = pd.concat(tmp)

In [ ]:
os.remove("switch-inputs/variable_capacity_factors.tab")
variable_tab = variable_cap.groupby('GENERATION_PROJECT')
for keys in variable_tab.groups.keys():
    data = variable_tab.get_group(keys).reset_index(drop=True)
    data.index +=1
    data.index.name = 'timepoint'
    data.rename(columns={'capacity_factor': 'gen_max_capacity_factor'},
               inplace=True)
    data.reset_index()[['GENERATION_PROJECT', 'timepoint', 'gen_max_capacity_factor']].to_csv('switch-inputs/variable_capacity_factors.tab', 
                                                                  sep='\t', index=False, 
                mode='a', header=(not os.path.exists('switch-inputs/variable_capacity_factors.tab')))

In [ ]:
loads_tmp = loads[loads.year <= 2025]
list_tmp = []
tmp = (loads_tmp.loc[output_data['date']].drop(['year', 'month','day','hour', 'total'], axis=1).reset_index()
        .drop_duplicates('index').reset_index(drop=True))
del tmp['index']
tmp = tmp.unstack(0)
for name, group in tmp.groupby(level=0):
    list_tmp.append(group.reset_index())

In [ ]:
loads_tab = pd.concat(list_tmp)
loads_tab;

In [ ]:
loads_tab = pd.concat(list_tmp)
loads_tab.index += 1
loads_tab = loads_tab.rename(columns={'level_0':'LOAD_ZONE', 0:'zone_demand_mw'})
del loads_tab['level_1']
loads_tab.index.name = 'TIMEPOINT'
loads_tab = loads_tab.reset_index()[['LOAD_ZONE', 'TIMEPOINT', 'zone_demand_mw']]
loads_tab.to_csv('switch-inputs/loads.tab', sep='\t', index=False)
loads_tab;